In [6]:
import pandas as pd
import numpy as np
# nhập bảng prices
prices = pd.read_csv('prices.csv').sort_values(by = ['product_id','updated_at'], ascending = True)
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
# nhập bảng sales
sales = pd.read_csv('sales.csv').sort_values(by = ['product_id','ordered_at'], ascending = True)
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
# merge_asof 2 bảng với sales là bảng trái
uptodate_prices = pd.merge_asof(sales.sort_values('ordered_at'), prices.sort_values('updated_at'), 
                           by = 'product_id', 
                           left_on = 'ordered_at', 
                           right_on = 'updated_at', 
                           direction = 'nearest')
# tạo cột nếu have_new_prices['ordered_at'] < have_new_prices['updated_at'] thì true
uptodate_prices['have_old_prices'] = uptodate_prices['ordered_at'] < uptodate_prices['updated_at']
# tạo cột revenue
uptodate_prices['revenue'] = np.where(uptodate_prices['have_old_prices'] == False,
                                     uptodate_prices['quantity_ordered']*uptodate_prices['new_price'],
                                    uptodate_prices['quantity_ordered']*uptodate_prices['old_price'])

In [7]:
uptodate_prices.head()

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,have_old_prices,revenue
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,False,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,True,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,True,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,True,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,False,53500


In [9]:
#Tổng Revenue theo từng product_id
uptodate_prices.groupby(['product_id'])['revenue'].sum().reset_index()

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500
